Here is Notebook for find more similar jobs to you skills, and find skill set gab between your skills and job skills.

I used word2vec for feature engineering.
Then use n_similarity between inserted skills and all Dataset skills after remove all nan and deplicated rows.

In [2]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
import string
import ast
from nltk.stem.wordnet import WordNetLemmatizer
lemma = WordNetLemmatizer()
# Import Word2Vec
import gensim
from gensim.models import Word2Vec


In [22]:
#read data
data=pd.read_csv('jobs_skills.csv')
# delete some columns
data = data.drop(['jobFunction','industry','_id'],axis=1)
# delete nan rows
data=data[data['skills'] != "['nan']"]
# delete duplicated rows
data=data.drop_duplicates(subset ="skills")
# clean job title by remove everything after (_)
data['title']=data['title'].str.split('-').str[0]

# clean job skills
def clean(row):
    row=row.lower()
    row_list=ast.literal_eval(row)
    lemma_text= [" ".join(lemma.lemmatize(word) for word in sentence.split()) for sentence in row_list]
    return lemma_text

data['skills']= [clean(row) for row in data['skills']]
data.head()

,title,skills
0,Property Consultant,"[sale, retail, real estate, sale target, indoo..."
1,Sales Representative,"[sale, real estate, sale target, customer serv..."
2,Receptionist,"[admin work, office management, administration..."
3,Senior Property Consultant,"[sale, real estate, sale target, computer skil..."
4,Senior SEO Specialist,"[marketing campaign, e-marketing, digital mark..."


In [37]:
#use word2vec to build word emmbedding for all skills
trained_model = Word2Vec(data['skills'], size =50, window = 3, min_count=1,
sg = 1)
# save model
trained_model.save('trained_model.bin')
# load model
trained_model = Word2Vec.load('trained_model.bin')

In [70]:
#fuction take list of skills and return more similar job titles and skill set gap
def find_similarity(input_skills):
    similarity_list=[]
    #n_similarity between input_skills and all data skills and append it to list
    for _,row in data.iterrows():
        similarity_list.append(trained_model.n_similarity(row[1], input_skills))
        
    similarity_series = pd.Series(similarity_list).sort_values(ascending = False)
    
    # getting the indexes of the 10 most similar jobs
    top_10_indexes = list(similarity_series.iloc[0:10].index)
    
    # Build Dataframe of more similar jobs to inserted skills
    s_data = ([data.iloc[i,0],similarity_series.iloc[i] ,list(set(data['skills'][0])-set(data.iloc[i,1]))] 
              for i in top_10_indexes) 
    df = pd.DataFrame(s_data , columns = ['job title','Similarity' ,'Skill Set Gap'])
    return df   

In [71]:
jobs_similar_to_skills= find_similarity(['sales', 'retail', 'real estate', 'sales target', 'indoor sales', 'sales skills', 'property sales'])
print(jobs_similar_to_skills)

C:\Users\DINA\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  """


                     job title  Similarity  \
0          Property Consultant    1.000000   
1         Sales Representative    0.996162   
2         Property Consultant     0.999848   
3   Senior Property Consultant    0.780005   
4  Junior Property Consultant     0.993862   
5    Indirect Sales Executive     0.995626   
6    Commercial Sales Manager     0.793453   
7         Property Consultant     0.999798   
8                  Sales Agent    0.991490   
9      Senior Property Advisor    0.993149   

                            Skill Set Gap  
0                                      []  
1                                [retail]  
2                                [retail]  
3                                [retail]  
4                [property sales, retail]  
5  [property sales, indoor sales, retail]  
6          [property sales, indoor sales]  
7  [property sales, indoor sales, retail]  
8  [property sales, indoor sales, retail]  
9                  [retail, sales target]  
